<a href="https://colab.research.google.com/github/AWH-GlobalPotential-X/AWH-Geo/blob/master/notebooks/AWH-Geo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to AWH-Geo

This tool requires a [Google Drive](https://drive.google.com/drive/my-drive) and [Earth Engine](https://developers.google.com/earth-engine/) Account.

[Start here](https://drive.google.com/drive/u/1/folders/1EzuqsbADrtdXChcpHqygTh7SuUw0U_QB) to create a new Output Table from the template:
 1. Right-click on "OutputTable_TEMPLATE" file > Make a Copy to your own Drive folder
 2. Rename the new file "OuputTable_CODENAME" with CODENAME (max 83 characters!) as a unique output table code. If including a date in the code, use the YYYYMMDD date format.
 3. Enter in the output values in L/hr to each cell in each of the 10%-interval rH bins... interpolate in Sheets as necessary.

Then, click "Connect" at the top right of this notebook.

Then run each of the code blocks below, following instructions. For "OutputTableCode" inputs, use the CODENAME you created in Sheets.




In [ ]:
#@title Basic setup and earthengine access.

print('Welcome to AWH-Geo')

# import, authenticate, then initialize EarthEngine module ee
# https://developers.google.com/earth-engine/python_install#package-import
import ee 
print('Make sure the EE version is v0.1.215 or greater...')
print('Current EE version = v' + ee.__version__)
print('')
ee.Authenticate()
ee.Initialize()

worldGeo = ee.Geometry.Polygon( # Created for some masking and geo calcs
  coords=[[-180,-90],[-180,0],[-180,90],[-30,90],[90,90],[180,90],
          [180,0],[180,-90],[30,-90],[-90,-90],[-180,-90]],
  geodesic=False,
  proj='EPSG:4326'
)




In [ ]:
#@title Test Earth Engine connection (see Mt Everest elev and a green map)
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

# Access study assets
from IPython.display import Image
jmpGeofabric_image = ee.Image('users/awhgeoglobal/jmpGeofabric_image') # access to study folder in EE
Image(url=jmpGeofabric_image.getThumbUrl({'min': 0, 'max': 1, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))


In [ ]:
#@title Set up access to Google Sheets (follow instructions)
from google.colab import auth
auth.authenticate_user()

# gspread is module to access Google Sheets through python
# https://gspread.readthedocs.io/en/latest/index.html
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default()) # get credentials

In [ ]:
#@title STEP 1: Export timeseries for given OutputTable: enter CODENAME (without "OutputTable_" prefix) below

OutputTableCode = "" #@param {type:"string"}
StartYear = 2010 #@param {type:"integer"}
EndYear = 2020 #@param {type:"integer"}
ExportWeekly_1_or_0 =   0#@param {type:"integer"}

ee_username = ee.String(ee.Dictionary(ee.List(ee.data.getAssetRoots()).get(0)).get('id'))
ee_username = ee_username.getInfo()

years = list(range(StartYear,EndYear))
print('Time Period: ', years)

def timeseriesExport(outputTable_code):
  
  """
  This script runs the output table value over the climate variables using the 
  nearest lookup values, worldwide, every three hours during a user-determined 
  period. It then resamples the temporal interval by averaging the hourly output 
  over semi-week periods. It then converts the resulting image collection into a 
  single image with several bands, each of which representing one (hourly or 
  semi-week) interval. Finally, it exports this image over 3-month tranches and 
  saves each as an EE Image Assets with appropriate names corresponding to the 
  tranche's time period. 
  """
  
  # print the output table code from user input for confirmation
  print('outputTable code:', outputTable_code)

  # CLIMATE DATA PRE-PROCESSING
  # ERA5-Land climate dataset used for worldwide (derived) climate metrics
  # https://www.ecmwf.int/en/era5-land
  # era5-land HOURLY images in EE catalog
  era5Land = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') 
  # print('era5Land',era5Land.limit(50)) # print some data for inspection (debug)
  era5Land_proj = era5Land.first().projection() # get ERA5-Land projection & scale for export
  era5Land_scale = era5Land_proj.nominalScale()
  print('era5Land_scale (should be ~11132):',era5Land_scale.getInfo())
  era5Land_filtered = era5Land.filterDate( # ERA5-Land climate data
    str(StartYear-1) + '-12-31', str(EndYear) + '-01-01').select( # filter by date
        # filter by ERA5-Land image collection bands                              
        [
         'dewpoint_temperature_2m', # K (https://apps.ecmwf.int/codes/grib/param-db?id=168)
         'surface_solar_radiation_downwards', # J/m^2 (Accumulated value. Divide by 3600 to get W/m^2 over hourly interval https://apps.ecmwf.int/codes/grib/param-db?id=176)
         'temperature_2m' # K
         ]) 
  # print('era5Land_filtered',era5Land_filtered.limit(50))

  print('Wait... retrieving data from sheets takes a couple minutes')

  # COLLECT OUTPUT TABLE DATA FROM SHEETS INTO PYTHON ARRAYS
  # gspread function which will look in list of gSheets accessible to user
  # in Earth Engine, an array is a list of lists.
  # loop through worksheet tabs and build a list of lists of lists (3 dimensional)
  # to organize output values [L/hr] by the 3 physical variables in the following
  # order: by temperature (first nesting leve), ghi (second nesting level), then
  # rH (third nesting level).
  spreadsheet = gc.open('OutputTable_' + outputTable_code) 
  outputArray = list() # create empty array
  rH_labels = ['rH0','rH10','rH20','rH30','rH40','rH50', # worksheet tab names
               'rH60','rH70','rH80','rH90','rH100']
  for rH in rH_labels: # loop to create 3-D array (list of lists of lists)
    rH_interval_array = list() 
    worksheet = spreadsheet.worksheet(rH)
    for x in list(range(7,26)): # relevant ranges in output table sheet
      rH_interval_array.append([float(y) for y in worksheet.row_values(x)])
    outputArray.append(rH_interval_array)
  # print('Output Table values:', outputArray) # for debugging
  # create an array image in EE (each pixel is a multi-dimensional matrix)
  outputImage_arrays = ee.Image(ee.Array(outputArray)) # values are in [L/hr]

  def processTimeseries(i): # core processing algorithm with lookups to outputTable

    """
    This is the core AWH-Geo algorithm to convert image-based input climate data 
    into an image of AWG device output [L/time] based on a given output lookup table.
    It runs across the ERA5-Land image collection timeseries and runs the lookup table 
    on each pixel of each image representing each hourly climate timestep.
    """

    i = ee.Image(i) # cast as image
    i = i.updateMask(i.select('temperature_2m').mask()) # ensure mask is applied to all bands
    timestamp_millis = ee.Date(i.get('system:time_start'))
    i_previous = ee.Image(era5Land_filtered.filterDate(
        timestamp_millis.advance(-1,'hour')).first())
    rh = ee.Image().expression( # relative humidity calculation [%]
    # from http://bmcnoldy.rsmas.miami.edu/Humidity.html
      '100 * (e**((17.625 * Td) / (To + Td)) / e**((17.625 * T) / (To + T)))', {
        'e': 2.718281828459045, # Euler's constant
        'T': i.select('temperature_2m').subtract(273.15), # temperature K converted to Celsius [°C]
        'Td': i.select('dewpoint_temperature_2m').subtract(273.15), # dewpoint temperature K converted to Celsius [°C]
        'To': 243.04 # reference temperature [K]
      }).rename('rh')
    ghi = ee.Image(ee.Algorithms.If( # because this parameter in ERA5 is cumulative in J/m^2...
        condition=ee.Number(timestamp_millis.get('hour')).eq(1), # ...from last obseration...
        trueCase=i.select('surface_solar_radiation_downwards'), # ...current value must be...
        falseCase=i.select('surface_solar_radiation_downwards').subtract( # ...subtracted from last...
            i_previous.select('surface_solar_radiation_downwards')) # ... then divided by seconds
      )).divide(3600).rename('ghi') # solar global horizontal irradiance [W/m^2]
    temp = i.select('temperature_2m'
                    ).subtract(273.15).rename('temp') # temperature K converted to Celsius [°C]
    rhClamp = rh.clamp(0.1,100) # relative humdity clamped to output table range [%]
    ghiClamp = ghi.clamp(0.1,1300) # global horizontal irradiance clamped to range [W/m^2]
    tempClamp = temp.clamp(0.1,45) # temperature clamped to output table range [°C]
    # convert climate variables to lookup integers
    rhLookup = rhClamp.divide(10
                    ).round().int().rename('rhLookup') # rH lookup interval
    tempLookup = tempClamp.divide(2.5
                    ).round().int().rename('tempLookup') # temp lookup interval
    ghiLookup = ghiClamp.divide(100
                    ).add(1).round().int().rename('ghiLookup') # ghi lookup interval
    # combine lookup values in a 3-band image
    xyzLookup = ee.Image(rhLookup).addBands(tempLookup).addBands(ghiLookup) 
    # lookup values in 3D array for each pixel to return AWG output from table [L/hr]
    # set output to 0 if temperature is less than 0 deg C
    output = outputImage_arrays.arrayGet(xyzLookup).multiply(temp.gt(0))
    nightMask = ghi.gt(0.5) # mask pixels which have no incident sunlight

    return ee.Image(output.rename('O').addBands( # return image of output labeled "O" [L/hr]
      rh.updateMask(nightMask)).addBands(
          ghi.updateMask(nightMask)).addBands(
              temp.updateMask(nightMask)).setMulti({ # add physical variables as bands
        'system:time_start': timestamp_millis # set time as property
      })).updateMask(1) # close partial masks at continental edges

  def outputHourly_export(timeStart, timeEnd, year):

    """
    Run the lookup processing function (from above) across the entire climate 
    timeseries at the finest temporal interval (1 hr for ERA5-Land). Convert the 
    resulting image collection as a single image with a band for each timestep 
    to allow for export as an Earth Engine asset (you cannot export/save image
    collections as assets).
    """

    # filter ERA5-Land climate data by time
    era5Land_filtered_section = era5Land_filtered.filterDate(timeStart, timeEnd)

    # print('era5Land_filtered_section',era5Land_filtered_section.limit(1).getInfo())

    outputHourly = era5Land_filtered_section.map(processTimeseries) 
    # outputHourly_toBands_pre = outputHourly.select(['ghi']).toBands()
    outputHourly_toBands_pre = outputHourly.select(['O']).toBands()
    outputHourly_toBands = outputHourly_toBands_pre.select(
      # input climate variables as multiband image with each band representing timestep
      outputHourly_toBands_pre.bandNames(), 
      # rename bands by timestamp
      outputHourly_toBands_pre.bandNames().map(
        lambda name: ee.String('H').cat( # "H" for hourly
          ee.String(name).replace('T','')
        )
      )
    )

    # notify user of export
    print('Exporting outputHourly year:', year)
    task = ee.batch.Export.image.toAsset(
      image=ee.Image(outputHourly_toBands),
      region=worldGeo,
      description='O_hourly_' + outputTable_code + '_' + year,
      assetId=ee_username + '/O_hourly_' + outputTable_code + '_' + year,
      scale=era5Land_scale.getInfo(),
      maxPixels=1e10,
      maxWorkers=2000
    )
    task.start()
  
  # run timeseries export on entire hourly ERA5-Land for each yearly tranche

  for y in years:
    y = str(y)
    outputHourly_export(y + '-01-01', y + '-04-01', y + 'a')
    outputHourly_export(y + '-04-01', y + '-07-01', y + 'b')
    outputHourly_export(y + '-07-01', y + '-10-01', y + 'c')
    outputHourly_export(y + '-10-01', str(int(y)+1) + '-01-01', y + 'd')

  def outputWeekly_export(timeStart, timeEnd, year):

    era5Land_filtered_section = era5Land_filtered.filterDate(timeStart, timeEnd) # filter ERA5-Land climate data by time
    
    outputHourly = era5Land_filtered_section.map(processTimeseries)
    
    # resample values over time by 2-week aggregations
    # Define a time interval
    start = ee.Date(timeStart)
    end = ee.Date(timeEnd)
    # Number of years, in DAYS_PER_RANGE-day increments.
    DAYS_PER_RANGE = 14
    # DateRangeCollection, which contains the ranges we're interested in.
    drc = ee.call("BetterDateRangeCollection",
      start,
      end, 
      DAYS_PER_RANGE, 
      "day",
      True)
    # This filter will join images with the date range that contains their start time.
    filter = ee.Filter.dateRangeContains("date_range", None, "system:time_start")
    # Save all of the matching values under "matches".
    join = ee.Join.saveAll("matches")
    # Do the join.
    joinedResult = join.apply(drc, outputHourly, filter)
    # print('joinedResult',joinedResult)
    
    # Map over the functions, and add the mean of the matches as "meanForRange".
    joinedResult = joinedResult.map(
      lambda e: e.set("meanForRange", ee.ImageCollection.fromImages(e.get("matches")).mean())
    )
    # print('joinedResult',joinedResult)

    # roll resampled images into new image collection
    outputWeekly = ee.ImageCollection(joinedResult.map(
        lambda f: ee.Image(f.get('meanForRange'))
    ))
    # print('outputWeekly',outputWeekly.getInfo())

    # convert image collection into image with many bands which can be saved as EE asset
    outputWeekly_toBands_pre = outputWeekly.toBands()
    outputWeekly_toBands = outputWeekly_toBands_pre.select(
        outputWeekly_toBands_pre.bandNames(), # input climate variables as multiband image with each band representing timestep
        outputWeekly_toBands_pre.bandNames().map(
            lambda name: ee.String('W').cat(name)
        )
    )
    
    task = ee.batch.Export.image.toAsset(
      image=ee.Image(outputWeekly_toBands),
      region=worldGeo,
      description='O_weekly_' + outputTable_code + '_' + year,
      assetId=ee_username + '/O_weekly_' + outputTable_code + '_' + year,
      scale=era5Land_scale.getInfo(),
      maxPixels=1e10,
      maxWorkers=2000
    )
    if ExportWeekly_1_or_0 == 1:
      task.start() # remove comment hash if weekly exports are desired
      print('Exporting outputWeekly year:', year)

  # run semi-weekly timeseries export on ERA5-Land by year
  for y in years:
    y = str(y)
    outputWeekly_export(y + '-01-01', y + '-04-01', y + 'a')
    outputWeekly_export(y + '-04-01', y + '-07-01', y + 'b')
    outputWeekly_export(y + '-07-01', y + '-10-01', y + 'c')
    outputWeekly_export(y + '-10-01', str(int(y)+1) + '-01-01', y + 'd')

timeseriesExport(OutputTableCode)

print('Complete! Read instructions below')

# *Before moving on to the next step... Wait until above tasks are complete in the task manager: https://code.earthengine.google.com/*
(right pane, tab "tasks", click "refresh"; the should show up once the script prints "Exporting...")



In [ ]:
#@title Re-instate earthengine access (follow instructions)

print('Welcome Back to AWH-Geo')
print('')

# import, authenticate, then initialize EarthEngine module ee
# https://developers.google.com/earth-engine/python_install#package-import
import ee 
print('Make sure the EE version is v0.1.215 or greater...')
print('Current EE version = v' + ee.__version__)
print('')
ee.Authenticate()
ee.Initialize()

worldGeo = ee.Geometry.Polygon( # Created for some masking and geo calcs
  coords=[[-180,-90],[-180,0],[-180,90],[-30,90],[90,90],[180,90],
          [180,0],[180,-90],[30,-90],[-90,-90],[-180,-90]],
  geodesic=False,
  proj='EPSG:4326'
)



In [ ]:
#@title STEP 2: Export statistical results for given OutputTable: enter CODENAME (without "OutputTable_" prefix) below

ee_username = ee.String(ee.Dictionary(ee.List(ee.data.getAssetRoots()).get(0)).get('id'))
ee_username = ee_username.getInfo()

OutputTableCode = "" #@param {type:"string"}
StartYear = 2010 #@param {type:"integer"}
EndYear =  2020 #@param {type:"integer"}
SuffixName_optional = "" #@param {type:"string"}
ExportMADP90s_1_or_0 =  0#@param {type:"integer"}

years = list(range(StartYear,EndYear))
print('Time Period: ', years)

def generateStats(outputTable_code):
  
  """
  This function generates single images which contain time-aggregated output 
  statistics including overall mean and shortfall metrics such as MADP90s. 
  """
  
  # CLIMATE DATA PRE-PROCESSING
  # ERA5-Land climate dataset used for worldwide (derived) climate metrics
  # https://www.ecmwf.int/en/era5-land
  # era5-land HOURLY images in EE catalog
  era5Land = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') 
  # print('era5Land',era5Land.limit(50)) # print some data for inspection (debug)
  era5Land_proj = era5Land.first().projection() # get ERA5-Land projection & scale for export
  era5Land_scale = era5Land_proj.nominalScale()
  
  # setup the image collection timeseries to chart
  # unravel and concatenate all the image stages into a single image collection
  
  def unravel(i): # function to "unravel" image bands into an image collection
    def setDate(bandName): # loop over band names in image and return a LIST of ... 
      dateCode = ee.Date.parse( # ... images, one for each band
          format='yyyyMMddHH',
          date=ee.String(ee.String(bandName).split('_').get(0)).slice(1) # get date periods from band name
      )
      return i.select([bandName]).rename('O').set('system:time_start',dateCode)
    i = ee.Image(i)
    return i.bandNames().map(setDate) # returns a LIST of images

  yearCode_list = ee.List(sum([[ # each image units in [L/hr]
      unravel(ee.Image(ee_username + '/O_hourly_' + outputTable_code + '_' + str(y)+'a')),
      unravel(ee.Image(ee_username + '/O_hourly_' + outputTable_code + '_' + str(y)+'b')),
      unravel(ee.Image(ee_username + '/O_hourly_' + outputTable_code + '_' + str(y)+'c')),
      unravel(ee.Image(ee_username + '/O_hourly_' + outputTable_code + '_' + str(y)+'d'))              
  ] for y in years], [])).flatten()

  outputTimeseries = ee.ImageCollection(yearCode_list)

  Od_overallMean = outputTimeseries.mean().multiply(24).rename('Od') # hourly output x 24 = mean daily output [L/day]
  
  # export overall daily mean
  task = ee.batch.Export.image.toAsset(
    image=Od_overallMean,
    region=worldGeo,
    description='Od_overallMean_' + outputTable_code + SuffixName_optional,
    assetId=ee_username + '/Od_overallMean_' + outputTable_code + SuffixName_optional,
    scale=era5Land_scale.getInfo(),
    maxPixels=1e10,
    maxWorkers=2000
  )
  task.start()
  print('Exporting Od_overallMean_' + outputTable_code + SuffixName_optional)

  ## run the moving average function over the timeseries using DAILY averages

  # start and end dates over which to calculate aggregate statistics
  startDate = ee.Date(str(StartYear) + '-01-01')
  endDate = ee.Date(str(EndYear) + '-01-01')

  # resample values over time by daily aggregations
  # Number of years, in DAYS_PER_RANGE-day increments.
  DAYS_PER_RANGE = 1
  # DateRangeCollection, which contains the ranges we're interested in.
  drc = ee.call('BetterDateRangeCollection',
    startDate,
    endDate, 
    DAYS_PER_RANGE, 
    'day',
    True)
  # This filter will join images with the date range that contains their start time.
  filter = ee.Filter.dateRangeContains('date_range', None, 'system:time_start')
  # Save all of the matching values under "matches".
  join = ee.Join.saveAll('matches')
  # Do the join.
  joinedResult = join.apply(drc, outputTimeseries, filter)
  # print('joinedResult',joinedResult)
  
  # Map over the functions, and add the mean of the matches as "meanForRange".
  joinedResult = joinedResult.map(
    lambda e: e.set('meanForRange', ee.ImageCollection.fromImages(e.get('matches')).mean())
  )
  # print('joinedResult',joinedResult)

  # roll resampled images into new image collection
  outputDaily = ee.ImageCollection(joinedResult.map(
      lambda f: ee.Image(f.get('meanForRange')).set(
        'system:time_start',
        ee.Date.parse('YYYYMMdd',f.get('system:index')).millis()
      )
  ))
  # print('outputDaily',outputDaily.getInfo())

  outputDaily_p90 = ee.ImageCollection( # collate rolling periods into new image collection of rolling average values
      outputDaily.toList(outputDaily.size())).reduce(
        ee.Reducer.percentile( # reduce image collection by percentile
          [10] # 100% - 90% = 10%
        )).multiply(24).rename('Od')

  task = ee.batch.Export.image.toAsset(
    image=outputDaily_p90,
    region=worldGeo,
    description='Od_DailyP90_' + outputTable_code + SuffixName_optional,
    assetId=ee_username + '/Od_DailyP90_' + outputTable_code + SuffixName_optional,
    scale=era5Land_scale.getInfo(),
    maxPixels=1e10,
    maxWorkers=2000
  )
  if ExportMADP90s_1_or_0 == 1:
    task.start()
    print('Exporting Od_DailyP90_' + outputTable_code + SuffixName_optional)

  def rollingStats(period): # run rolling stat function for each rolling period scenerio

    # collect neighboring time periods into a join
    timeFilter = ee.Filter.maxDifference(
      difference=float(period)/2 * 24 * 60 * 60 * 1000, # mid-centered window
      leftField='system:time_start', 
      rightField='system:time_start'
    )
    rollingPeriod_join = ee.ImageCollection(ee.Join.saveAll('images').apply(
      primary=outputDaily, # apply the join on itself to collect images
      secondary=outputDaily, 
      condition=timeFilter
    ))
    def rollingPeriod_mean(i): # get the mean across each collected periods
      i = ee.Image(i) # collected images stored in "images" property of each timestep image
      return ee.ImageCollection.fromImages(i.get('images')).mean()
    outputDaily_rollingMean = rollingPeriod_join.filterDate(
        startDate.advance(float(period)/2,'days'),
        endDate.advance(float(period)/-2,'days')
    ).map(rollingPeriod_mean,True)

    Od_p90_rolling = ee.ImageCollection( # collate rolling periods into new image collection of rolling average values
      outputDaily_rollingMean.toList(outputDaily_rollingMean.size())).reduce(
        ee.Reducer.percentile( # reduce image collection by percentile
          [10] # 100% - 90% = 10%
        )).multiply(24).rename('Od') # hourly output x 24 = mean daily output [L/day]

    task = ee.batch.Export.image.toAsset(
      image=Od_p90_rolling,
      region=worldGeo,
      description='Od_MADP90_'+ period + 'day_' + outputTable_code + SuffixName_optional,
      assetId=ee_username + '/Od_MADP90_'+ period + 'day_' + outputTable_code + SuffixName_optional,
      scale=era5Land_scale.getInfo(),
      maxPixels=1e10,
      maxWorkers=2000
    )
    if ExportMADP90s_1_or_0 == 1:
      task.start()
      print('Exporting Od_MADP90_' + period + 'day_' + outputTable_code + SuffixName_optional)
  
  rollingPeriods = [
                    '007',
                    '030',
                    # '060',
                    '090',
                    # '180',
                    ] # define custom rolling periods over which to calc MADP90 [days]
        
  for period in rollingPeriods: # execute the calculations & export
    # print(period)
    rollingStats(period)

generateStats(OutputTableCode) # run stats function

print('Complete! Go to next step.')

Wait until these statistics are completed processing. Track them in the task manager: https://code.earthengine.google.com/

When they are finished.... [Go here to see maps](https://code.earthengine.google.com/fac0cc72b2ac2e431424cbf45b2852cf)